In [88]:
# нью рок нью джаз
import numpy as np
import pandas as pd

In [89]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

imputer = SimpleImputer(strategy="most_frequent")

In [90]:
path_train = "train"

wag_prob = pd.read_parquet(path_train + "/wagons_probeg_ownersip.parquet").convert_dtypes()
target = pd.read_csv(path_train +'/target/y_train.csv').convert_dtypes()
to_predict = pd.read_csv(path_train +'/target/y_predict.csv').convert_dtypes()

# добавляем kri_izm
kri_izm = pd.read_parquet(path_train+'/kti_izm.parquet').convert_dtypes()

In [91]:
target.head()

,wagnum,month,target_month,target_day
0,33361,2023-01-01,0,0
1,33364,2023-01-01,0,0
2,33366,2023-01-01,0,0
3,33358,2023-01-01,0,0
4,33349,2023-01-01,0,0


### Предобработка основной таблицы с вагонами

In [0]:
train_target = target[target["month"] == "2023-01-01"].drop("month", axis=1)
train = wag_prob[wag_prob["repdate"] < "2023-01-01"]

features = train.groupby("wagnum").agg(
    {
        "ost_prob": ["mean", "last", "max"],
        "manage_type": "last",
        "rod_id": "last",
        "reestr_state": "last",
    }
)
features.columns = ["_".join(col).strip() for col in features.columns.values]
features = features.reset_index()


### Предобработка параметров вагонов

In [ ]:

wag_param = pd.read_parquet(path_train + "/wag_params.parquet").convert_dtypes()

m_train = wag_param.drop(
    ["model", "tipvozd", "date_build", "srok_sl", "date_iskl"], axis=1
)

### Предобработка "геометрии" вагонов

In [108]:
kti_izm = pd.read_parquet('train/kti_izm.parquet', parse_dates=['operation_date']).convert_dtypes()
kti_izm.head()

,wagnum,operation_date_dttm,mileage_all,axl1_l_w_flange,axl1_r_w_flange,axl2_l_w_flange,axl2_r_w_flange,axl3_l_w_flange,axl3_r_w_flange,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,325,2022-08-01,112091,30.5,31.0,26.7,27.4,28.8,28.6,26.3,27.4,67.5,65.6,66.7,64.5,61.5,60.3,67.9,66.8
1,325,2022-08-03,112471,30.4,31.4,27.5,27.6,28.9,29.0,25.5,27.8,65.7,65.8,65.7,63.7,61.6,61.4,67.4,69.3
2,325,2022-08-05,113938,30.0,31.1,27.1,27.9,28.5,28.9,26.2,27.6,70.8,69.5,65.1,65.6,62.2,60.7,65.7,69.6
3,325,2022-09-15,121071,30.5,31.4,26.4,27.3,28.9,28.4,24.9,28.0,70.2,68.1,67.4,65.8,62.3,63.0,70.9,70.3
4,325,2022-09-09,117341,30.7,31.5,26.9,27.1,28.9,28.5,25.5,27.5,68.1,64.1,64.5,64.9,61.6,61.1,68.7,<NA>


In [ ]:
kti_izm_agg = kti_izm.groupby("wagnum").agg({
    
})

In [0]:


merged = features.merge(m_train, on="wagnum", how="inner")
merged = train_target.merge(merged, on="wagnum", how="left")


In [92]:

column_names = merged.columns

merged = imputer.fit_transform(merged)

merged = pd.DataFrame(merged, columns=column_names)

X_train = merged.drop(["wagnum"], axis=1)
y_month_name = "target_month"
y_day_name = "target_day"
target_names = [y_month_name, y_day_name]

In [93]:
print("Training for month and day prediction")

logreg_month = LogisticRegression()
logreg_month.fit(X_train.drop(target_names, axis=1), X_train[y_month_name])
logreg_day = LogisticRegression()
logreg_day.fit(X_train.drop(target_names, axis=1), X_train[y_day_name])

rf_month = RandomForestClassifier()
rf_month.fit(X_train.drop(target_names, axis=1), X_train[y_month_name])
rf_day = RandomForestClassifier()
rf_day.fit(X_train.drop(target_names, axis=1), X_train[y_day_name])

cb_month = CatBoostClassifier()
cb_month.fit(X_train.drop(target_names, axis=1), X_train[y_month_name])
cb_day = CatBoostClassifier()
cb_day.fit(X_train.drop(target_names, axis=1), X_train[y_day_name])

Training for month and day prediction
Learning rate set to 0.046422
0:	learn: 0.5909363	total: 10ms	remaining: 10s
1:	learn: 0.5114632	total: 17.8ms	remaining: 8.87s
2:	learn: 0.4503527	total: 24.6ms	remaining: 8.17s
3:	learn: 0.3812159	total: 30.9ms	remaining: 7.7s
4:	learn: 0.3438650	total: 36.6ms	remaining: 7.29s
5:	learn: 0.3051889	total: 42.4ms	remaining: 7.03s
6:	learn: 0.2728274	total: 48.2ms	remaining: 6.84s
7:	learn: 0.2401121	total: 54.1ms	remaining: 6.71s
8:	learn: 0.2263839	total: 59.5ms	remaining: 6.55s
9:	learn: 0.2030350	total: 65.5ms	remaining: 6.48s
10:	learn: 0.1841495	total: 71.6ms	remaining: 6.44s
11:	learn: 0.1697516	total: 77.8ms	remaining: 6.41s
12:	learn: 0.1616439	total: 83.7ms	remaining: 6.36s
13:	learn: 0.1515778	total: 89.7ms	remaining: 6.32s
14:	learn: 0.1432438	total: 95ms	remaining: 6.24s
15:	learn: 0.1363699	total: 100ms	remaining: 6.16s
16:	learn: 0.1303176	total: 106ms	remaining: 6.13s
17:	learn: 0.1261611	total: 112ms	remaining: 6.09s
18:	learn: 0.122

In [94]:
path_test = 'test'
test_target = pd.read_csv(path_test+'/target/y_test.csv').convert_dtypes().drop("month", axis=1)
#test = wag_prob[wag_prob["repdate"] >= "2023-01-01"]
test = wag_prob

In [95]:
# test_target = target[target["month"] == "2023-01-01"].drop("month", axis=1)
# test = wag_prob[wag_prob["repdate"] < "2023-01-01"]

test_features = test.groupby("wagnum").agg(
    {
        "ost_prob": ["mean", "last", "max"],
        "manage_type": "last",
        "rod_id": "last",
        "reestr_state": "last",
    }
)
test_features.columns = ["_".join(col).strip() for col in test_features.columns.values]
test_features = test_features.reset_index()

m_test = wag_param.drop(
    ["model", "tipvozd", "date_build", "srok_sl", "date_iskl"], axis=1
)

merged = test_features.merge(m_test, on="wagnum", how="inner")
merged = test_target.merge(merged, on="wagnum", how="left")
column_names = merged.columns
merged = imputer.fit_transform(merged)
merged = pd.DataFrame(merged, columns=column_names)

X_test = merged.drop(["wagnum"], axis=1)

In [96]:
y_pred_logreg_month = logreg_month.predict(X_test.drop(target_names, axis=1))
y_pred_rf_month = rf_month.predict(X_test.drop(target_names, axis=1))
y_pred_cb_month = cb_month.predict(X_test.drop(target_names, axis=1))

y_pred_logreg_day = logreg_day.predict(X_test.drop(target_names, axis=1))
y_pred_rf_day = rf_day.predict(X_test.drop(target_names, axis=1))
y_pred_cb_day = cb_day.predict(X_test.drop(target_names, axis=1))

print("Month score:")
f1_logreg_month = f1_score(X_test[y_month_name], y_pred_logreg_month)
print(f"Logistic Regression month F1 Score: {f1_logreg_month}")
f1_rf_month = f1_score(X_test[y_month_name], y_pred_rf_month)
print(f"Random Forest month F1 Score: {f1_rf_month}")
f1_cb_month = f1_score(X_test[y_month_name], y_pred_cb_month)
print(f"CatBoost month F1 Score: {f1_cb_month}")

print("Day score:")
f1_logreg_day = f1_score(X_test[y_day_name], y_pred_logreg_day)
print(f"Logistic Regression day F1 Score: {f1_logreg_day}")
f1_rf_day = f1_score(X_test[y_day_name], y_pred_rf_day)
print(f"Random Forest day F1 Score: {f1_rf_day}")
f1_cb_day = f1_score(X_test[y_day_name], y_pred_cb_day)
print(f"CatBoost day F1 Score: {f1_cb_day}")

Month score:
Logistic Regression month F1 Score: 0.21640664318067435
Random Forest month F1 Score: 0.48169418521177315
CatBoost month F1 Score: 0.4966300106420715
Day score:
Logistic Regression day F1 Score: 0.0
Random Forest day F1 Score: 0.1989100817438692
CatBoost day F1 Score: 0.2777777777777778


In [97]:
def correct_days_with_month(day_array, month_array):
    corrected_day = day_array.copy()
    corrected_day[month_array == 0] = 0
    return corrected_day


y_pred_logreg_day_corrected = correct_days_with_month(y_pred_logreg_day, y_pred_logreg_month)
y_pred_rf_day_corrected = correct_days_with_month(y_pred_rf_day, y_pred_rf_month)
y_pred_cb_day_corrected = correct_days_with_month(y_pred_cb_day, y_pred_cb_month)


print("Day score after applying month info to day:")
f1_logreg_day = f1_score(X_test[y_day_name], y_pred_logreg_day_corrected)
print(f"Logistic Regression day F1 Score: {f1_logreg_day}")
f1_rf_day = f1_score(X_test[y_day_name], y_pred_rf_day_corrected)
print(f"Random Forest day F1 Score: {f1_rf_day}")
f1_cb_day = f1_score(X_test[y_day_name], y_pred_cb_day_corrected)
print(f"CatBoost day F1 Score: {f1_cb_day}")


Day score after applying month info to day:
Logistic Regression day F1 Score: 0.0
Random Forest day F1 Score: 0.18707015130674
CatBoost day F1 Score: 0.2692778457772338


In [98]:
y_pred_logreg_day

array([0., 0., 0., ..., 0., 0., 0.])

In [99]:
X_test.drop(target_names, axis=1).columns

Index(['ost_prob_mean', 'ost_prob_last', 'ost_prob_max', 'manage_type_last',
       'rod_id_last', 'reestr_state_last', 'rod_id', 'gruz',
       'cnsi_gruz_capacity', 'cnsi_volumek', 'tara', 'zavod_build',
       'cnsi_probeg_dr', 'cnsi_probeg_kr', 'kuzov', 'telega', 'tormoz',
       'tippogl', 'norma_km', 'ownertype'],
      dtype='object')

In [100]:
feature_importances = cb_month.get_feature_importance()
feature_names = X_test.drop(target_names, axis=1).columns
importance_dict = {feature_names[i]: importance for i, importance in enumerate(feature_importances)}
importance_dict

{'ost_prob_mean': 20.660371509121738,
 'ost_prob_last': 16.926915998252227,
 'ost_prob_max': 16.50418151737648,
 'manage_type_last': 0.3664173764967447,
 'rod_id_last': 2.786355892331363,
 'reestr_state_last': 0.009104263123337307,
 'rod_id': 1.4516671015410596,
 'gruz': 7.357776269918021,
 'cnsi_gruz_capacity': 3.687731885539861,
 'cnsi_volumek': 4.135742999151402,
 'tara': 7.472033782716449,
 'zavod_build': 3.82797133231902,
 'cnsi_probeg_dr': 2.6595080525433445,
 'cnsi_probeg_kr': 0.25400800847348043,
 'kuzov': 2.438243585815221,
 'telega': 0.9397971746070224,
 'tormoz': 2.7797642264806974,
 'tippogl': 3.7156110006187837,
 'norma_km': 1.6878933744052358,
 'ownertype': 0.3389046491684878}

In [101]:
# import os
# import h2o
# import xgboost as xgb
# import pandas as pd
# from h2o.automl import H2OAutoML
# 
# h2o.init()
# 
# model_path = ""
# model_file = os.path.join(model_path, "StackedEnsemble_AllModels_1_AutoML_2_20231111_112553")
# train = h2o.H2OFrame(X_train)
# test = h2o.H2OFrame(X_test)
# train[y_name] = train[y_name].asfactor()
# test[y_name] = test[y_name].asfactor()
# 
# x = train.columns
# y = y_name
# x.remove(y)
#     
# # if os.path.exists(model_file):
# #     print("sdfsd")
# #     aml = h2o.load_model(model_file)
# # else:
# aml = H2OAutoML(max_models=15, seed=42)
# aml.train(x=x, y=y, training_frame=train)
# 
# h2o.save_model(aml.leader, path=model_path)
# 
# lb = aml.leaderboard
# print(lb.head(rows=lb.nrows))

In [102]:
# y_pred = aml.predict(test).as_data_frame().squeeze()
# result = f1_score(X_test[y_name], y_pred["predict"])
# print(f"F1: {result}")


In [103]:
# h2o.save_model(aml.leader, path=f"saved_models/h20_{result}")
# h2o.shutdown()

In [104]:
# feature_importance = logreg.coef_
# 
# avg_feature_importance = np.mean(np.abs(feature_importance), axis=0)
# 
# feature_importance_df = pd.DataFrame(
#     {"Feature": X_train.columns, "Importance": avg_feature_importance}
# )
# 
# feature_importance_df = feature_importance_df.sort_values(
#     by="Importance", ascending=False
# )
# 
# print(feature_importance_df)

In [105]:
# feature_importance = rf.feature_importances_
# 
# feature_importance_df = pd.DataFrame(
#     {"Feature": X_train.columns, "Importance": feature_importance}
# )
# 
# feature_importance_df = feature_importance_df.sort_values(
#     by="Importance", ascending=False
# )
# 
# print(feature_importance_df)

In [106]:
# import shap
# 
# explainer = shap.TreeExplainer(rf)
# shap_values = explainer.shap_values(X_train)
# shap.summary_plot(shap_values, X_train, plot_type="bar")